# Imports

In [268]:
import sys

modules_to_reload = [
    "src.utils.method_loggers",
    "src.utils.method_runners",
    "src.utils.metrics_calculators",
    "src.utils.tensor_handlers",
    "src.utils.trackers",
    "src.utils.video_controller",
    "src.utils.optimal_rank_finders",
]

for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

%load_ext memory_profiler
%load_ext autoreload
%autoreload 2

import gc
import os

import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
import tensorflow as tf
import torch
from tqdm import tqdm
from scipy.signal import argrelextrema

np.random.seed(42)
os.environ["OPENBLAS_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"

tf.random.set_seed(42)

import time
from itertools import product

import tensorly as tl
from dotenv import load_dotenv
from scipy.optimize import minimize

from src.utils.image_controller import download_image, extract_image_frames
from src.utils.metrics_calculators import IMetricCalculator
from src.utils.optimal_rank_finders import (
    find_optimal_rank_tensor_train_by_compression_ratio,
)

load_dotenv()

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

# get tensor

In [269]:
cache_dir_image = "../.cache/image"

image_urls = [
    # "https://i.pinimg.com/564x/04/b2/68/04b26838bdd5e2ba54d0144558685bae.jpg",
    "https://cdnstatic.rg.ru/crop620x412/uploads/images/187/94/47/iStock-644032024.jpg",
    # "https://i.sstatic.net/uQggz.png",
]

images = {}

In [270]:
image_paths = [download_image(image_url, cache_dir_image) for image_url in image_urls]

Изображение уже загружено и закешировано: ../.cache/image/iStock-644032024.jpg


In [271]:
for image_index, image_path in enumerate(image_paths):
    image_frames = extract_image_frames(image_path)

    images[f"image-{image_index}"] = {
        "image_url": image_urls[image_index],
        "image_path": image_path,
        "frames": image_frames,
    }

    print(f"iamge-{image_index} - {image_frames.shape}")

iamge-0 - (412, 620, 3)


In [272]:
tensor_train_args = {"svd": "truncated_svd"}

In [273]:
example_tensor = images["image-0"]["frames"].copy().astype(np.float32)

# func for calculate bounds for tensor train factors

In [274]:
def calculate_tt_bounds(shape: tuple | list) -> list:
    """
    Calculates the bounds for TT-ranks of a tensor based on its shape.

    Parameters
    ----------
    shape : tuple or list
        List or tuple of tensor dimensions. Each element represents the size of the tensor along that dimension.

    Returns
    -------
    list
        List of rank bounds in the format [(1, 1), (1, r1_max), ..., (1, 1)].

    """
    d = len(shape)
    bounds = [(1, 1)]

    for k in range(1, d):
        prod_left = 1
        for i in range(k):
            prod_left *= shape[i]

        prod_right = 1
        for j in range(k, d):
            prod_right *= shape[j]

        rk_max = min(prod_left, prod_right)
        bounds.append((1, rk_max))

    bounds.append((1, 1))
    return bounds

In [275]:
tt_bounds_example_tensor = calculate_tt_bounds(example_tensor.shape)

In [276]:
example_tensor.shape

(412, 620, 3)

In [277]:
tt_bounds_example_tensor

[(1, 1), (1, 412), (1, 3), (1, 1)]

# func for calculate optimal initial rank of tensor train

In [278]:
# def calculate_tensor_train_initial_rank(bounds: tuple) -> list[int]:
#     return [max(1, round(max_bound / 2)) for min_bound, max_bound in bounds]

def calculate_tensor_train_initial_rank(bounds: tuple) -> list[int]:
    return [min_bound for min_bound, max_bound in bounds]

In [279]:
example_tensor_initial_rank = calculate_tensor_train_initial_rank(tt_bounds_example_tensor)

In [280]:
example_tensor_initial_rank

[1, 1, 1, 1]

# calculate metrics for the entire search area for example tensor

In [281]:
target_compression_ratio_for_graphs = 50.0
frobenius_error_coef_for_graphs = 1.0
compression_ratio_coef_for_graphs = 10.0

In [282]:
rank_ranges = [range(bound[0], bound[1] + 1) for bound in tt_bounds_example_tensor]

In [231]:
search_area_example_results = {}

for rank_combination in tqdm(
    product(*rank_ranges), total=np.prod([len(r) for r in rank_ranges]), desc="Processing Ranks"
):
    test_rank = list(rank_combination)
    internal_indices = test_rank[1:-1]

    try:
        with tl.backend_context("pytorch"):
            example_tensor_cuda = tl.tensor(example_tensor).to("cuda")
            method_result = tl.decomposition.tensor_train(example_tensor_cuda, rank=test_rank, **tensor_train_args)
            tt_factors = method_result
            reconstructed_tensor = tl.tt_to_tensor(tt_factors)

            frobenius_error = (
                tl.norm(reconstructed_tensor - example_tensor_cuda) / tl.norm(example_tensor_cuda)
            ).item()
            compression_ratio = IMetricCalculator.get_tensors_size(*tt_factors) / IMetricCalculator.get_tensors_size(
                example_tensor_cuda
            )
            compression_penalty = (target_compression_ratio_for_graphs / 100 - compression_ratio) ** 2
            loss_function_result = (
                frobenius_error_coef_for_graphs * frobenius_error
                + compression_ratio_coef_for_graphs * compression_penalty
            )

            search_area_example_results[tuple(internal_indices)] = {
                "rank": test_rank,
                "frobenius_error": frobenius_error,
                "compression_ratio": compression_ratio,
                "compression_penalty": compression_penalty,
                "loss_function_result": loss_function_result,
            }

            del tt_factors, reconstructed_tensor
    except Exception as e:
        search_area_example_results[tuple(internal_indices)] = {"rank": test_rank, "error": str(e)}
    finally:
        torch.cuda.empty_cache()
        gc.collect()

Processing Ranks: 100%|██████████| 2067/2067 [09:51<00:00,  3.50it/s]


# Optimization algs test

## optimize with custom algorithm

In [232]:
start_time = time.perf_counter()
best_rank, compression_ratio, frobenius_error, find_rank_logs = find_optimal_rank_tensor_train_by_compression_ratio(
    tensor=example_tensor,
    target_compression_ratio=50.0,
    tensor_train_args=tensor_train_args,
    search_strategy="custom",
)
elapsed_time = time.perf_counter() - start_time

Optimal rank search process for TensorTrain:
step | rank | compression ratio (%) | frobenius error (%)
1 | [1, 2, 1, 1] | 0.152668 % | 40.194961 %
2 | [1, 2, 2, 1] | 0.249548 % | 25.621590 %
3 | [1, 3, 2, 1] | 0.374201 % | 22.926879 %
4 | [1, 3, 3, 1] | 0.519460 % | 8.690936 %
5 | [1, 4, 3, 1] | 0.692492 % | 6.977586 %
6 | [1, 5, 3, 1] | 0.865524 % | 6.360334 %
7 | [1, 6, 3, 1] | 1.038556 % | 5.764327 %
8 | [1, 7, 3, 1] | 1.211588 % | 5.361496 %
9 | [1, 8, 3, 1] | 1.384619 % | 5.057926 %
10 | [1, 9, 3, 1] | 1.557651 % | 4.793444 %
11 | [1, 10, 3, 1] | 1.730683 % | 4.531646 %
12 | [1, 11, 3, 1] | 1.903715 % | 4.337652 %
13 | [1, 12, 3, 1] | 2.076747 % | 4.144634 %
14 | [1, 13, 3, 1] | 2.249779 % | 3.987119 %
15 | [1, 14, 3, 1] | 2.422811 % | 3.841257 %
16 | [1, 15, 3, 1] | 2.595843 % | 3.700722 %
17 | [1, 16, 3, 1] | 2.768875 % | 3.558835 %
18 | [1, 17, 3, 1] | 2.941906 % | 3.425606 %
19 | [1, 18, 3, 1] | 3.114938 % | 3.308032 %
20 | [1, 19, 3, 1] | 3.287970 % | 3.201620 %
21 | [1, 20, 

In [233]:
print(
    f"Tensor shape = {list(example_tensor.shape)}",
    f"Best Rank = {best_rank}",
    f"Frobenius Error = {frobenius_error:.6f}%",
    f"Compression Ratio = {compression_ratio:.6f}%",
    f"Elapsed Time = {elapsed_time:.6f} seconds",
    sep="\n",
)

# Tensor shape = [564, 564, 3]
# Best Rank = [1, 212, 3, 1]
# Frobenius Error = 0.893109%
# Compression Ratio = 50.119146%
# Elapsed Time = 55.487687 seconds

Tensor shape = [689, 1195, 3]
Best Rank = [1, 289, 3, 1]
Frobenius Error = 0.010288%
Compression Ratio = 50.006579%
Elapsed Time = 79.026014 seconds


## scipy algs

### Funcs for check scipy algs

In [234]:
def loss_function(
    rank: list,
    tensor: np.ndarray,
    target_compression_ratio: float,
    tensor_train_args: dict[str, str],
    frobenius_error_coef: float = 1.0,
    compression_ratio_coef: float = 10.0,
):
    try:
        tt_factors = tl.decomposition.tensor_train(tensor, rank=rank, **tensor_train_args)
        reconstructed_tensor = tl.tt_to_tensor(tt_factors)

        frobenius_error = (tl.norm(reconstructed_tensor - tensor) / tl.norm(tensor)).item()
        compression_ratio = IMetricCalculator.get_tensors_size(*tt_factors) / IMetricCalculator.get_tensors_size(tensor)

        target_compression_ratio /= 100

        compression_penalty = (target_compression_ratio - compression_ratio) ** 2

        # compression_penalty = target_compression_ratio - compression_ratio
        #
        # if compression_ratio > 1.0 or compression_ratio < 0.0 or compression_penalty < 0.0 or compression_penalty > 1.0:
        #     compression_penalty = float("inf")

        return frobenius_error_coef * frobenius_error + compression_ratio_coef * compression_penalty

    except Exception:
        return float("inf")

In [235]:
def optimize_rank(
    tensor: np.ndarray,
    target_compression_ratio: float,
    tensor_train_args: dict[str, str],
    initial_rank: list[int],
    frobenius_error_coef: float = 1.0,
    compression_ratio_coef: float = 2.0,
    optimization_method: str = "nelder-mead",
    jac: str | None = None,
    hess: str | None = None,
):
    def loss_wrapper(free_rank: list):
        full_rank = [1] + list(np.clip(np.round(free_rank).astype(int), 1, None)) + [1]  # noqa: RUF005
        return loss_function(
            rank=full_rank,
            tensor=tensor,
            target_compression_ratio=target_compression_ratio,
            tensor_train_args=tensor_train_args,
            frobenius_error_coef=frobenius_error_coef,
            compression_ratio_coef=compression_ratio_coef,
        )

    def calculate_tt_bounds(tensor_shape: tuple | list) -> list:
        """
        Calculates the bounds for TT-ranks of a tensor based on its shape.

        Parameters
        ----------
        tensor_shape : tuple or list
            List or tuple of tensor dimensions. Each element represents the size of the tensor along that dimension.

        Returns
        -------
        list
            List of rank bounds in the format [(1, 1), (1, r1_max), ..., (1, 1)].

        """
        d = len(tensor_shape)
        bounds = [(1, 1)]

        for k in range(1, d):
            prod_left = 1
            for i in range(k):
                prod_left *= tensor_shape[i]

            prod_right = 1
            for j in range(k, d):
                prod_right *= tensor_shape[j]

            rk_max = min(prod_left, prod_right)
            bounds.append((1, rk_max))

        bounds.append((1, 1))
        return bounds

    class OptimizationLogger:
        def __init__(self):
            self.logs = []
            self.current_iteration = -1

        def callback(self, xk):
            self.current_iteration += 1
            rank = [1] + list(np.round(xk).astype(int)) + [1] # noqa: RUF005
            self.logs.append(
                {
                    "step": self.current_iteration,
                    "rank": rank,
                    "loss_evals": [],
                    "raw_xk": xk,
                }
            )
            print(f"\n=== Iteration {self.current_iteration} complete ===", f"New rank estimate: {rank}\n", sep="\n")

    optimization_logger = OptimizationLogger()

    free_rank = initial_rank[1:-1]

    # params
    is_bounds_variable_usable = [
        "nelder-mead",
        "l-bfgs-b",
        "tnc",
        "slsqp",
        "powell",
        "trust-constr",
        "cobyla",
        "cobyqa",
    ]

    is_adaptive_variable_usable = ["nelder-mead"]

    is_jac_variable_usable = [
        "cg",
        "bfgs",
        "newton-cg",
        "l-bfgs-b",
        "tnc",
        "slsqp",
        "trust-ncg",
        "trust-krylov",
        "trust-exact",
        "trust-constr",
    ]

    is_hess_variable_usable = ["newton-cg", "dogleg", "trust-ncg", "trust-krylov", "trust-exact", " trust-constr"]

    is_callback_variable_not_usable = ["tnc", "slsqp", "cobyla"]

    free_bounds = calculate_tt_bounds(tensor.shape)[1:-1] if optimization_method in is_bounds_variable_usable else None

    callback_param = (
        optimization_logger.callback if optimization_method not in is_callback_variable_not_usable else None
    )

    adaptive = optimization_method.lower() in is_adaptive_variable_usable

    jac = jac if optimization_method.lower() in is_jac_variable_usable else None

    hess = hess if optimization_method.lower() in is_hess_variable_usable else None

    minimize_kwargs = {
        "fun": loss_wrapper,
        "x0": free_rank,
        "method": optimization_method,
        "jac": jac,
        "hess": hess,
        "bounds": free_bounds,
        "callback": callback_param,
        "options": {
            "disp": True,
            # "maxiter": 1000,
        },
    }

    if adaptive:
        minimize_kwargs["options"]["adaptive"] = adaptive

    if jac:
        minimize_kwargs["jac"] = jac

    if hess:
        minimize_kwargs["hess"] = hess

    # params

    result = minimize(**minimize_kwargs)

    optimal_rank = [1] + list(np.clip(np.round(result.x).astype(int), 1, None)) + [1]  # noqa: RUF005
    final_loss = result.fun

    return optimal_rank, final_loss, result, optimization_logger.logs

### args for algs

In [236]:
example_tensor_initial_rank

[1, 1, 1, 1]

### check algs

https://docs.scipy.org/doc/scipy-1.15.0/tutorial/optimize.html#

#### nelder-mead

In [237]:
method = "nelder-mead"
frobenius_error_coef = 1.0
compression_ratio_coef = 10.0

In [238]:
# print(
#     f"Testing optimization method: {method}",
#     f"Tensor shape: {example_tensor.shape}",
#     f"Initial rank: {example_tensor_initial_rank}",
#     sep="\n",
# )
# try:
#     # check optimizer method
#     start_time = time.perf_counter()
#     optimal_rank, final_loss, minimize_result_nelder_mead, iteration_logs_nelder_mead = optimize_rank(
#         tensor=example_tensor,
#         target_compression_ratio=50.0,
#         tensor_train_args=tensor_train_args,
#         initial_rank=example_tensor_initial_rank,
#         optimization_method=method,
#         frobenius_error_coef=frobenius_error_coef,
#         compression_ratio_coef=compression_ratio_coef,
#     )
#     elapsed_time = time.perf_counter() - start_time
# 
#     # check final frobenius error and compression ratio
#     tt_factors = tl.decomposition.tensor_train(example_tensor, rank=optimal_rank, **tensor_train_args)
#     reconstructed_tensor = tl.tt_to_tensor(tt_factors)
# 
#     frobenius_error = 100.0 * (tl.norm(reconstructed_tensor - example_tensor) / tl.norm(example_tensor)).item()
#     compression_ratio = (
#         100.0 * IMetricCalculator.get_tensors_size(*tt_factors) / IMetricCalculator.get_tensors_size(example_tensor)
#     )
# 
#     print(
#         f"Optimal rank: {optimal_rank}",
#         f"Elapsed time: {elapsed_time:.6f} seconds",
#         f"Frobenius Error: {frobenius_error:.6f}%",
#         f"Compression Ratio: {compression_ratio:.6f}%",
#         sep="\n",
#         end="\n\n",
#     )
# except Exception as e:
#     print(f"Error with method {method}: {e}")

In [239]:
# minimize_result_nelder_mead

In [240]:
# iteration_logs_nelder_mead

#### powell

In [283]:
method = "powell"
frobenius_error_coef = 1.0
compression_ratio_coef = 10.0

In [284]:
print(
    f"Testing optimization method: {method}",
    f"Tensor shape: {example_tensor.shape}",
    f"Initial rank: {example_tensor_initial_rank}",
    sep="\n",
)
try:
    # check optimizer method
    start_time = time.perf_counter()
    optimal_rank, final_loss, minimize_result_powell, iteration_logs_powell = optimize_rank(
        tensor=example_tensor,
        target_compression_ratio=50.0,
        tensor_train_args=tensor_train_args,
        initial_rank=example_tensor_initial_rank,
        optimization_method=method,
        frobenius_error_coef=frobenius_error_coef,
        compression_ratio_coef=compression_ratio_coef,
    )
    elapsed_time = time.perf_counter() - start_time

    # check final frobenius error and compression ratio
    tt_factors = tl.decomposition.tensor_train(example_tensor, rank=optimal_rank, **tensor_train_args)
    reconstructed_tensor = tl.tt_to_tensor(tt_factors)

    frobenius_error = 100.0 * (tl.norm(reconstructed_tensor - example_tensor) / tl.norm(example_tensor)).item()
    compression_ratio = (
        100.0 * IMetricCalculator.get_tensors_size(*tt_factors) / IMetricCalculator.get_tensors_size(example_tensor)
    )

    print(
        f"Optimal rank: {optimal_rank}",
        f"Elapsed time: {elapsed_time:.6f} seconds",
        f"Frobenius Error: {frobenius_error:.6f}%",
        f"Compression Ratio: {compression_ratio:.6f}%",
        sep="\n",
        end="\n\n",
    )
except Exception as e:
    print(f"Error with method {method}: {e}")

Testing optimization method: powell
Tensor shape: (412, 620, 3)
Initial rank: [1, 1, 1, 1]

=== Iteration 0 complete ===
New rank estimate: [1, 371, 2, 1]


=== Iteration 1 complete ===
New rank estimate: [1, 233, 2, 1]


=== Iteration 2 complete ===
New rank estimate: [1, 233, 2, 1]

Optimization terminated successfully.
         Current function value: 0.034328
         Iterations: 3
         Function evaluations: 126
Optimal rank: [1, 233, 2, 1]
Elapsed time: 31.024402 seconds
Frobenius Error: 3.427543%
Compression Ratio: 50.229930%



In [243]:
# minimize_result_powell

In [244]:
# iteration_logs_powell

#### SLSQP

In [245]:
method = "slsqp"
frobenius_error_coef = 1.0
compression_ratio_coef = 10.0

In [246]:
# print(
#     f"Testing optimization method: {method}",
#     f"Tensor shape: {example_tensor.shape}",
#     f"Initial rank: {example_tensor_initial_rank}",
#     sep="\n",
# )
# try:
#     # check optimizer method
#     start_time = time.perf_counter()
#     optimal_rank, final_loss, minimize_result_slsqp, iteration_logs_slsqp = optimize_rank(
#         tensor=example_tensor,
#         target_compression_ratio=50.0,
#         tensor_train_args=tensor_train_args,
#         initial_rank=example_tensor_initial_rank,
#         optimization_method=method,
#         frobenius_error_coef=frobenius_error_coef,
#         compression_ratio_coef=compression_ratio_coef,
#         jac=None,
#     )
#     elapsed_time = time.perf_counter() - start_time
# 
#     # check final frobenius error and compression ratio
#     tt_factors = tl.decomposition.tensor_train(example_tensor, rank=optimal_rank, **tensor_train_args)
#     reconstructed_tensor = tl.tt_to_tensor(tt_factors)
# 
#     frobenius_error = 100.0 * (tl.norm(reconstructed_tensor - example_tensor) / tl.norm(example_tensor)).item()
#     compression_ratio = (
#         100.0 * IMetricCalculator.get_tensors_size(*tt_factors) / IMetricCalculator.get_tensors_size(example_tensor)
#     )
# 
#     print(
#         f"Optimal rank: {optimal_rank}",
#         f"Elapsed time: {elapsed_time:.6f} seconds",
#         f"Frobenius Error: {frobenius_error:.6f}%",
#         f"Compression Ratio: {compression_ratio:.6f}%",
#         sep="\n",
#         end="\n\n",
#     )
# except Exception as e:
#     print(f"Error with method {method}: {e}")

In [247]:
# minimize_result_slsqp

In [248]:
# iteration_logs_slsqp

# some graphs

## search area example

In [249]:
internal_indices = np.array(list(search_area_example_results.keys()))
metrics = ["frobenius_error", "compression_ratio", "compression_penalty", "loss_function_result"]

metric_dict = {tuple(idx): search_area_example_results[idx] for idx in search_area_example_results}

figs = []

for metric in metrics:
    z_values = np.array([search_area_example_results[key].get(metric, np.nan) for key in search_area_example_results])
    x_indices = internal_indices[:, 0]
    y_indices = internal_indices[:, 1]

    local_min_points = []

    for i, (x, y) in enumerate(zip(x_indices, y_indices)):
        z = z_values[i]

        neighbors = [
            (x - 1, y), (x + 1, y),
            (x, y - 1), (x, y + 1)
        ]

        is_local_min = all(
            (neighbor not in metric_dict or metric_dict[neighbor].get(metric, np.inf) >= z)
            for neighbor in neighbors
        )

        if is_local_min:
            local_min_points.append((x, y, z))

    x_min, y_min, z_min = zip(*local_min_points) if local_min_points else ([], [], [])

    fig = go.Figure()

    fig.add_trace(go.Scatter3d(
        x=x_indices,
        y=y_indices,
        z=z_values,
        mode="markers",
        marker={"size": 5, "color": z_values, "colorscale": "Viridis", "opacity": 0.8},
        name=metric
    ))

    fig.add_trace(go.Scatter3d(
        x=x_min,
        y=y_min,
        z=z_min,
        mode="markers+text",
        marker={"size": 6, "color": "red", "symbol": "diamond"},
        text=[f"min: {val:.6f}" for val in z_min],
        textposition="top center",
        name="Local Minima"
    ))

    fig.update_layout(
        title=f"Search area for example tensor of {metric.replace('_', ' ').title()}",
        scene={
            "xaxis_title": "Rank Index 1",
            "yaxis_title": "Rank Index 2",
            "zaxis_title": metric.replace("_", " ").title(),
            "yaxis": {"tickmode": "array", "tickvals": list(set(y_indices.astype(int)))}
        },
        margin={"l": 0, "r": 0, "t": 40, "b": 0},
        template="plotly_white",
        showlegend=False,
    )

    figs.append(fig)

In [250]:
html_str = ""
for fig in figs:
    html_str += go.Figure(fig).to_html(full_html=False, include_plotlyjs=False)

html_file = f"""
<!DOCTYPE html>
<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
<h1>Search area by some metrics</h1>
{html_str}
</body>
</html>
"""

output_path = "../.cache/data_analyze/optimization_algs_for_tensor_train_search_area.html"
with open(output_path, "w", encoding="utf-8") as f:  # noqa: PTH123
    f.write(html_file)

## custom alg metrics

In [251]:
target_compression_ratio_for_graphs_percent = 50.0
frobenius_error_coef_for_graphs = 1.0
compression_ratio_coef_for_graphs = 10.0

In [252]:
(
    custom_alg_compression_ratios,
    custom_alg_frobenius_errors,
    custom_alg_compression_penalties,
    custom_alg_loss_function_results,
) = [], [], [], []
for element in find_rank_logs:
    compression_ratio = element["compression_ratio"] / 100.0
    frobenius_error = element["frobenius_error"] / 100.0
    target_compression_ratio_for_graphs = target_compression_ratio_for_graphs_percent / 100.0

    custom_alg_compression_ratios.append(compression_ratio)
    custom_alg_frobenius_errors.append(frobenius_error)

    compression_penalty = (target_compression_ratio_for_graphs - compression_ratio) ** 2
    loss_function_result = (
        frobenius_error_coef_for_graphs * frobenius_error + compression_ratio_coef_for_graphs * compression_penalty
    )

    custom_alg_compression_penalties.append(compression_penalty)
    custom_alg_loss_function_results.append(loss_function_result)

## path in search area

In [253]:
figs = []

for metric, metric_data in zip(
        metrics,
        [
            custom_alg_frobenius_errors,
            custom_alg_compression_ratios,
            custom_alg_compression_penalties,
            custom_alg_loss_function_results,
        ],
        strict=False,
):
    z_values = np.array([search_area_example_results[key].get(metric, np.nan) for key in search_area_example_results])
    x_indices = internal_indices[:, 0]
    y_indices = internal_indices[:, 1]

    # Поиск локальных минимумов
    local_min_points = []

    metric_dict = {tuple(idx): search_area_example_results[idx] for idx in search_area_example_results}

    for i, (x, y) in enumerate(zip(x_indices, y_indices)):
        z = z_values[i]

        neighbors = [
            (x - 1, y), (x + 1, y),  # По оси X
            (x, y - 1), (x, y + 1)   # По оси Y
        ]

        is_local_min = all(
            (neighbor not in metric_dict or metric_dict[neighbor].get(metric, np.inf) >= z)
            for neighbor in neighbors
        )

        if is_local_min:
            local_min_points.append((x, y, z))

    x_min, y_min, z_min = zip(*local_min_points) if local_min_points else ([], [], [])

    fig = go.Figure()

    # Основные точки
    fig.add_trace(go.Scatter3d(
        x=x_indices,
        y=y_indices,
        z=z_values,
        mode="markers",
        marker={"size": 5, "color": z_values, "colorscale": "Viridis", "opacity": 0.8},
    ))

    path_x = []
    path_y = []
    path_z = []

    for i, log in enumerate(find_rank_logs):
        rank = log["rank"]
        if metric == "frobenius_error":
            z_value = custom_alg_frobenius_errors[i]
        elif metric == "compression_ratio":
            z_value = custom_alg_compression_ratios[i]
        elif metric == "compression_penalty":
            z_value = custom_alg_compression_penalties[i]
        elif metric == "loss_function_result":
            z_value = custom_alg_loss_function_results[i]

        path_x.append(rank[1])
        path_y.append(rank[2])
        path_z.append(z_value)

        fig.add_trace(
            go.Scatter3d(
                x=[rank[1]],
                y=[rank[2]],
                z=[z_value],
                mode="markers",
                marker={
                    "size": 10 if i == 0 or i == len(find_rank_logs) - 1 else 5,
                    "color": "yellow" if i == 0 or i == len(find_rank_logs) - 1 else "red",
                    "opacity": 0.8,
                },
            )
        )

    # Добавляем локальные минимумы (выделенные точки)
    fig.add_trace(go.Scatter3d(
        x=x_min,
        y=y_min,
        z=z_min,
        mode="markers+text",
        marker={"size": 8, "color": "blue", "symbol": "diamond"},
        text=[f"min: {val:.6f}" for val in z_min],
        textposition="top center",
    ))

    fig.add_trace(
        go.Scatter3d(
            x=path_x,
            y=path_y,
            z=path_z,
            mode="lines+markers",
            marker={"size": 5, "color": "red", "opacity": 0.8},
            line={"color": "red", "width": 3},
        )
    )

    fig.update_layout(
        title=f"Search area for example tensor of {metric.replace('_', ' ').title()} with custom alg path",
        scene={
            "xaxis_title": "Rank Index 1",
            "yaxis_title": "Rank Index 2",
            "zaxis_title": metric.replace("_", " ").title(),
            "yaxis": {"tickmode": "array", "tickvals": list(set(y_indices.astype(int)))},
        },
        margin={"l": 0, "r": 0, "t": 40, "b": 0},
        template="plotly_white",
        showlegend=False,
    )

    figs.append(fig)


In [254]:
html_str = ""
for fig in figs:
    html_str += go.Figure(fig).to_html(full_html=False, include_plotlyjs=False)

html_file = f"""
<!DOCTYPE html>
<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
<h1>Search area by some metrics</h1>
{html_str}
</body>
</html>
"""

output_path = "../.cache/data_analyze/optimization_algs_for_tensor_train_search_area_with_custom_alg.html"
with open(output_path, "w", encoding="utf-8") as f:  # noqa: PTH123
    f.write(html_file)

## only path

In [255]:
# figs = []
# 
# # График 1: Compression Ratios
# fig1 = go.Figure()
# fig1.add_trace(
#     go.Scatter(
#         x=list(range(len(custom_alg_compression_ratios))),
#         y=custom_alg_compression_ratios,
#         mode="lines+markers",
#         marker={"size": 5},
#         name="Compression Ratios",
#     )
# )
# fig1.update_layout(
#     title="Compression Ratios",
#     xaxis_title="Index",
#     yaxis_title="Values",
#     template="plotly_white",
#     margin={"l": 20, "r": 20, "t": 40, "b": 20},
# )
# figs.append(fig1)
# 
# # График 2: Frobenius Errors
# fig2 = go.Figure()
# fig2.add_trace(
#     go.Scatter(
#         x=list(range(len(custom_alg_frobenius_errors))),
#         y=custom_alg_frobenius_errors,
#         mode="lines+markers",
#         marker={"size": 5},
#         name="Frobenius Errors",
#     )
# )
# fig2.update_layout(
#     title="Frobenius Errors",
#     xaxis_title="Index",
#     yaxis_title="Values",
#     template="plotly_white",
#     margin={"l": 20, "r": 20, "t": 40, "b": 20},
# )
# figs.append(fig2)
# 
# # График 3: Compression Penalties
# fig3 = go.Figure()
# fig3.add_trace(
#     go.Scatter(
#         x=list(range(len(custom_alg_compression_penalties))),
#         y=custom_alg_compression_penalties,
#         mode="lines+markers",
#         marker={"size": 5},
#         name="Compression Penalties",
#     )
# )
# fig3.update_layout(
#     title="Compression Penalties",
#     xaxis_title="Index",
#     yaxis_title="Values",
#     template="plotly_white",
#     margin={"l": 20, "r": 20, "t": 40, "b": 20},
# )
# figs.append(fig3)
# 
# # График 4: Loss Function Results
# fig4 = go.Figure()
# fig4.add_trace(
#     go.Scatter(
#         x=list(range(len(custom_alg_loss_function_results))),
#         y=custom_alg_loss_function_results,
#         mode="lines+markers",
#         marker={"size": 5},
#         name="Loss Function Results",
#     )
# )
# fig4.update_layout(
#     title="Loss Function Results",
#     xaxis_title="Index",
#     yaxis_title="Values",
#     template="plotly_white",
#     margin={"l": 20, "r": 20, "t": 40, "b": 20},
# )
# figs.append(fig4)

In [256]:
# html_str = ""
# for i, fig in enumerate(figs):
#     html_str += pio.to_html(fig, full_html=False, include_plotlyjs=False)
# 
# html_file = f"""
# <!DOCTYPE html>
# <html>
# <head>
#     <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
# </head>
# <body>
# <h1>
# Custom alg metrics
# </h1>
# {html_str}
# </body>
# </html>
# """
# 
# with open("../.cache/data_analyze/optimization_algs_for_tensor_train_custom_alg.html", "w", encoding="utf-8") as f: #noqa: PTH123
#     f.write(html_file)

# optimize with scipy alg for tensor

In [257]:
# cache_dir_eeg = "../.cache/eeg"
#
# target_tensor = create_eeg_limo_data_tensor(cache_dir_eeg=cache_dir_eeg)

In [258]:
# target_initial_rank = calculate_tensor_train_initial_rank(target_tensor.shape)

In [259]:
# method = "powell"
# frobenius_error_coef = 1.0
# compression_ratio_coef = 10.0

In [260]:
# print(
#     f"Testing optimization method: {method}",
#     f"Tensor shape: {target_tensor.shape}",
#     f"Initial rank: {target_initial_rank}",
#     sep="\n",
# )
# try:
#     # check optimizer method
#     start_time = time.perf_counter()
#     optimal_rank, final_loss = optimize_rank(
#         tensor=target_tensor,
#         target_compression_ratio=50.0,
#         method_max_bound=max(target_tensor.shape),
#         tensor_train_args=tensor_train_args,
#         initial_rank=target_initial_rank,
#         optimization_method=method,
#         frobenius_error_coef=frobenius_error_coef,
#         compression_ratio_coef=compression_ratio_coef,
#     )
#     elapsed_time = time.perf_counter() - start_time
#
#     # check final frobenius error and compression ratio
#     tt_factors = tl.decomposition.tensor_train(target_tensor, rank=optimal_rank, **tensor_train_args)
#     reconstructed_tensor = tl.tt_to_tensor(tt_factors)
#
#     frobenius_error = 100.0 * (tl.norm(reconstructed_tensor - target_tensor) / tl.norm(target_tensor)).item()
#     compression_ratio = (
#         100.0 * IMetricCalculator.get_tensors_size(*tt_factors) / IMetricCalculator.get_tensors_size(target_tensor)
#     )
#
#     print(
#         f"Optimal rank: {optimal_rank}",
#         f"Elapsed time: {elapsed_time:.6f} seconds",
#         f"Frobenius Error: {frobenius_error:.6f}%",
#         f"Compression Ratio: {compression_ratio:.6f}%",
#         sep="\n",
#         end="\n\n",
#     )
# except Exception as e:
#     print(f"Error with method {method}: {e}")